In [1]:
import sys
sys.path.insert(0,'..')

In [13]:
!pip install -q git+https://github.com/huggingface/transformers.git


OSError: [Errno 12] Cannot allocate memory

In [3]:
!pip install -q datasets albumentations


In [2]:
# not sure if we actually need this
from datasets import load_dataset

import pandas as pd
import numpy as np
from PIL import Image
from emsim.geant.dataset import GeantElectronDataset
from transformers import MaskFormerImageProcessor


In [3]:
pixelated_file = '/home/zero/emsim/segmentation/pixelated_1pt25um_tracks_thinned_4um_back_20k_300keV.txt'
true_pixelated_file = '/home/zero/emsim/segmentation/true_pixelated_1pt25um_thinned_4um_back_20k_300keV.txt'

In [4]:
from transformers import MaskFormerImageProcessor

processor = MaskFormerImageProcessor(reduce_labels=True, ignore_index=255, do_resize=False, do_rescale=False, do_normalize=False)

/home/zero/anaconda3/lib/python3.10/site-packages/transformers/models/maskformer/image_processing_maskformer.py:428: FutureWarning: The `reduce_labels` argument is deprecated and will be removed in v4.27. Please use `do_reduce_labels` instead.
  warnings.warn(


In [5]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

train_transform = A.Compose([
    # A.Resize(width=512, height=512),
    A.Normalize(mean=0, std=1),
])

In [6]:
events_per_image_range = (12, 16)
dataset = GeantElectronDataset(pixels_file=pixelated_file, undiffused_file=true_pixelated_file, events_per_image_range=events_per_image_range, processor=processor, transform=train_transform)


In [7]:
print(dataset.transform)

Compose([
  Normalize(always_apply=False, p=1.0, mean=0, std=1, max_pixel_value=255.0),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True)


In [8]:
from torch.utils.data import DataLoader 
train_dataloader = DataLoader(dataset, batch_size=None)

In [9]:
from transformers import MaskFormerForInstanceSegmentation
from itertools import islice
from PIL import Image

current_chunk = islice(train_dataloader, 1)
# batch = next(current_chunk)

In [10]:
print(next(current_chunk))

ValueError: Unsupported number of image dimensions: 2

In [19]:
id2label = {id: 'electron' for id in range(50000)}
# Replace the head of the pre-trained model
# We specify ignore_mismatched_sizes=True to replace the already fine-tuned classification head by a new one
model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-base-ade",
                                                          id2label=id2label,
                                                          ignore_mismatched_sizes=True)

Some weights of MaskFormerForInstanceSegmentation were not initialized from the model checkpoint at facebook/maskformer-swin-base-ade and are newly initialized because the shapes did not match:
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([50001, 256]) in the model instantiated
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([50001]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([151]) in the checkpoint and torch.Size([50001]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import torch

def prep_inputs(chunk, processor, transform=None):
    image = np.array(chunk['image'].convert("RGB"))
    instance_seg = np.array(chunk['maps'][i].segmentation_map for i in range(len(batch['maps'])))

    if transform is not None:
        transformed = transform(image=image, mask=instance_seg)
        image, instance_seg = transformed['image'], transformed['mask']

        # "convert to C, H, W". wth does this do
        image = image.transpose(2, 0, 1)

    inputs = processor([image], [instance_seg], return_tensors="pt")
    inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}

    return inputs


    


In [21]:

import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

running_loss = 0.0
num_samples = 0
num_epochs = 100


for epoch in range(num_epochs):
  print("Epoch:", epoch)
  model.train()
  for idx, batch in enumerate(tqdm(train_dataloader)):
      # Reset the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(
              pixel_values=batch["pixel_values"].to(device),
              mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
              class_labels=[labels.to(device) for labels in batch["class_labels"]],
      )

      # Backward propagation
      loss = outputs.loss
      loss.backward()

      batch_size = batch["pixel_values"].size(0)
      running_loss += loss.item()
      num_samples += batch_size

      if idx % 100 == 0:
        print("Loss:", running_loss/num_samples)

      # Optimization
      optimizer.step()
     

Epoch: 0


0it [00:00, ?it/s]

ValueError: Height and Width of image, mask or masks should be equal. You can disable shapes check by setting a parameter is_check_shapes=False of Compose class (do it only if you are sure about your data consistency).